In [1]:
import MeCab
tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /Users/yuki/foo/bar/user_dic2.dic")
tagger.parse("")

'EOS\n'

In [2]:
from collections import Counter, defaultdict
import json
import re

In [3]:
def tokenize_ja(text):
    node = tagger.parseToNode(str(text)) #分かち書きしたのが入っている。イテレータ
    while node:
        surface = node.surface.lower() #surfaceは単語の見た目　例：トろ
        feature = node.feature.split(",")
        hinshi = feature[0]
        genkei = feature[6]
        if  surface != "" and hinshi in ["形容詞", "動詞", "副詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"]:
            yield genkei
        elif  surface != "" and hinshi == "名詞":
            yield surface
        node = node.next

In [4]:
def tokenize(content):
    return [token for token in tokenize_ja(content)]

In [5]:
text_dic = [['シャリは粒が小さめで美味しかったですが、ネタは不味かったです。', 0.9, -0.9],
 ['もう行きたくないです。', 0.8, -0.8],
 ['マグロ美味しい。', 0.7, 0.7],
 ['いくらも最高。', 0.3, 0.3],
 ['店長の傍にいたロシア系の美女は誰だろう。', 0, 0],
 ["マグロが不味かった。", 0.7, -0.9]]

In [6]:
word_dic = [["マグロ",2,["美味しい",1],["まずい",1]],
["美女",1,["誰",1],["ロシア",1]]]

In [16]:
def get_posinega(text_dic, json_file):
        f = open(json_file, "r")
        json_data = json.load(f)
        jiku_list = json_data["all_jiku"]["all_jiku_list"]
        #['赤酢', '握り', 'シャリ']
        positive_dic = defaultdict(float)
        negative_dic = defaultdict(float)
        result = {}
        for text in text_dic:
            t = tokenize(text[0])
            for jiku in jiku_list:
                jiku_group = json_data["all_jiku"][jiku]["jiku_group"]
                #['握り', 'にぎり', 'ニギリ']
                syusyoku_list = json_data["all_jiku"][jiku]["syusyoku"]["syusyoku_list"]
                #['大きい', '小さい', '創作']
                for ji in jiku_group:
                    if ji in t:
                        for syusyoku in syusyoku_list:
                            syusyoku_group = json_data["all_jiku"][jiku]["syusyoku"][syusyoku]
                            #[['大きい'], ['でかい'], ['大きめ'], ['ビッグ']]
                            for s in syusyoku_group:
                                if len(list(set(s) & set(t))) == len(s):
                                    if text[2] > 0:
                                        positive_dic[(jiku,syusyoku)] += text[1]*text[2]
                                    elif text[2] < 0:
                                        positive_dic[(jiku,syusyoku)] += -text[1]*text[2]
        for jiku in jiku_list:
            result[jiku] = []
            for p in positive_dic:
                
        return

In [19]:
get_posinega(text_dic, "../sample.json")

(defaultdict(float, {('シャリ', '小さい'): 0.81}), defaultdict(float, {}))

In [10]:
f

<_io.TextIOWrapper name='../sample.json' mode='r' encoding='UTF-8'>